<h1> Continuous 7 days login user

<h2>Method to solve problem
<h3>1.Remove duplicate day for each user
<h3>2.Give each  day a order number (1,2,3,4)
<h3>3.For continuous log in,each  day - order number =result ; rn / if continuous login the result rn will be same.
<h3>4.Group by user id , rn and filter >= 7

In [2]:
%load_ext sql


In [4]:
%%sql sqlite://
CREATE TABLE user_loginto(
    user_id int,
    login_time date);

INSERT INTO user_loginto(user_id,login_time)values
(1,'2025-01-01'),(1,'2025-01-01'),(1,'2025-01-02'),(1,'2025-01-03'),(1,'2025-01-05'),
(2,'2025-01-01'),(2,'2025-01-01'),(2,'2025-01-01'),
(3,'2025-01-01'),(3,'2025-01-02'),(3,'2025-01-03'),(3,'2025-01-04'),(3,'2025-01-05'),(3,'2025-01-06'),(3,'2025-01-07'),
(4,'2025-01-01'),(4,'2025-01-02'),(4,'2025-01-04'),(4,'2025-01-04'),(4,'2025-01-05'),(4,'2025-01-06'),(4,'2025-01-07');

INSERT INTO user_loginto(user_id,login_time)values
(4,'2025-01-01'),(4,'2025-01-02'),(4,'2025-01-04'),(4,'2025-01-04'),(4,'2025-01-05'),(4,'2025-01-06'),(4,'2025-01-07');
INSERT INTO user_loginto(user_id,login_time)values
(5,'2025-01-01'),(5,'2025-01-05'),(5,'2025-01-06'),(5,'2025-01-07');

Connecting to 'sqlite://'

22 rows affected.

7 rows affected.

4 rows affected.

++
||
++
++

In [6]:
%sql select * from user_loginto;

Running query in 'sqlite://'

user_id,login_time
1,2025-01-01
1,2025-01-01
1,2025-01-02
1,2025-01-03
1,2025-01-05
2,2025-01-01
2,2025-01-01
2,2025-01-01
3,2025-01-01
3,2025-01-02


In [76]:
%%sql
--step 1 Remove duplicate day for each user
WITH dedup AS (
    SELECT DISTINCT user_id, login_time
    FROM user_loginto
),
--step 2 Give each  day a order number (1,2,3,4)
ranked AS (
    SELECT 
        user_id,
        login_time,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY login_time) AS rn
    FROM dedup
),
--step 3 For continuous log in,each  day - order number =result ; rn / if continuous login the result rn will be same.
groupbed AS (
    SELECT 
        user_id,
        login_time,
        rn,
        DATE(login_time, '-' || rn || ' days') AS grp_date
    FROM ranked
),
--step 4 Group by user id , rn and filter >= 7
group_count AS(
    SELECT 
        user_id,
        count(*) as consecutive_cunt
    FROM groupbed
    group by user_id,grp_date
    )

SELECT * FROM group_count;


Running query in 'sqlite://'

user_id,consecutive_cunt
1,3
1,1
2,1
3,7
4,2
4,4
5,1
5,3


<h1>Each user first logint and last login time.

In [16]:
%%sql
select user_id,min(login_time) as first_login_time,max(login_time) as last_login_time
from user_loginto group by user_id;

Running query in 'sqlite://'

user_id,first_login_time,last_login_time
1,2025-01-01,2025-01-05
2,2025-01-01,2025-01-01
3,2025-01-01,2025-01-07
4,2025-01-01,2025-01-07
5,2025-01-01,2025-01-07
